In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/mt-upc/SHAS.git

In [ ]:
%cd /kaggle/working/

In [ ]:
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

In [ ]:
!./bin/micromamba create -y -p /kaggle/working/env -c conda-forge python=3.9.6

In [ ]:
!./bin/micromamba create -y -p /kaggle/working/env -c conda-forge python=3.9.6

In [ ]:
!./bin/micromamba run -p /kaggle/working/env python --version

In [ ]:
# Dùng pip trong env micromamba
!./bin/micromamba run -p /kaggle/working/env pip install torch==1.10.0 torchaudio==0.10.0
!./bin/micromamba run -p /kaggle/working/env pip install pandas==1.3.3 tqdm==4.62.3 numpy==1.22.1
!./bin/micromamba run -p /kaggle/working/env pip install transformers==4.11.3
!./bin/micromamba run -p /kaggle/working/env pip install sacrebleu==1.5.0 sacremoses==0.0.46
!./bin/micromamba run -p /kaggle/working/env pip install pydub==0.25.1 webrtcvad==2.0.10
!./bin/micromamba run -p /kaggle/working/env pip install wandb==0.12.9 SoundFile==0.10.3.post1
!./bin/micromamba run -p /kaggle/working/env pip install PyYAML==6.0 scikit_learn==1.0.2 tweepy==4.5.0
!./bin/micromamba run -p /kaggle/working/env pip install sentencepiece==0.1.96

In [ ]:
!mkdir models
!mkdir results

In [ ]:
%cd /kaggle/working/models

!git clone --branch shas --single-branch https://github.com/mt-upc/fairseq.git

In [ ]:
!mv /kaggle/working/models/fairseq /kaggle/working/models/fairseq_root

In [ ]:
!apt-get update
!apt-get install -y build-essential python3.9-dev
%cd /kaggle/working/
!./bin/micromamba run -p /kaggle/working/env pip install pip==23.3.2
!./bin/micromamba run -p /kaggle/working/env pip install --upgrade setuptools wheel cython numpy

In [ ]:
%cd /kaggle/working/models/fairseq_root
!/kaggle/working/bin/micromamba run -p /kaggle/working/env pip install --editable ./

In [ ]:
%cd /kaggle/working/models
! mkdir joint-s2t-multilingual

In [ ]:
# Tải các file model
files = ["checkpoint17.pt", "config.yaml", "tgt_dict.txt", "dict.txt", "spm.model"]
base_url = "https://dl.fbaipublicfiles.com/joint_speech_text_4_s2t/iwslt/iwslt_data"

for file in files:
    !wget {base_url}/{file} -O /kaggle/working/models/joint-s2t-multilingual/{file}

In [ ]:
%cd /kaggle/working/
!/kaggle/working/bin/micromamba run -p /kaggle/working/env pip install protobuf==3.20.3
!/kaggle/working/bin/micromamba run -p /kaggle/working/env pip install tensorboardX

# Gỡ numpy & pandas cũ (trong môi trường Python 3.9)
!/kaggle/working/bin/micromamba run -p /kaggle/working/env pip uninstall -y numpy pandas scipy

# Cài numpy phiên bản ổn định 
!/kaggle/working/bin/micromamba run -p /kaggle/working/env pip install numpy==1.22.1 pandas===1.3.3 scipy


In [ ]:
!wget https://dl.fbaipublicfiles.com/joint_speech_text_4_s2t/iwslt/iwslt_data/dict.txt -O /kaggle/working/results/dict.txt

In [ ]:
# Gán biến trong Python
mult_model_path = "/kaggle/working/models/joint-s2t-multilingual"
SHAS_ROOT = "/kaggle/working/SHAS"

# Chạy sed và python bằng shell: dùng f-string để truyền biến vào lệnh
!sed -i "s+/path/spm.model+{mult_model_path}/spm.model+" {mult_model_path}/config.yaml

%cd /kaggle/working/

!/kaggle/working/bin/micromamba run -p /kaggle/working/env python {SHAS_ROOT}/src/data_prep/fix_joint_s2t_cfg.py -c {mult_model_path}/checkpoint17.pt

In [ ]:
!cp /kaggle/input/segment/custom_segments.yaml /kaggle/working/results

In [ ]:
%cd /kaggle/working/
!/kaggle/working/bin/micromamba run -p /kaggle/working/env
with open("run_prepare_custom_dataset.py", "w") as f:
    f.write("""
import sys
import runpy

# Thêm đường dẫn đến fairseq và SHAS
sys.path.append('/kaggle/working/models/fairseq_root')
sys.path.append('/kaggle/working/SHAS/src/eval_scripts')

# Thiết lập sys.argv như gọi từ dòng lệnh
sys.argv = [
    "prepare_custom_dataset.py",
    "-y", "/kaggle/working/results/custom_segments.yaml",
    "-w", "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/wav",
    "-l", "en",
    "-i", "1"
]

# Chạy script như một module
runpy.run_module("prepare_custom_dataset", run_name="__main__")
""")
%cd /kaggle/working/
!/kaggle/working/bin/micromamba run -p /kaggle/working/env python run_prepare_custom_dataset.py


In [ ]:
%cd /kaggle/working/
with open("run_generate_translation.py", "w") as f:
    f.write("""
import sys
import os
import subprocess

sys.path.append("/kaggle/working/models/fairseq_root")

# Biến đầu vào
path_to_wavs = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/wav"
path_to_custom_segmentation_yaml = "/kaggle/working/results/custom_segments.yaml"
path_to_original_segmentation_yaml = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.yaml"
path_to_original_segment_transcriptions = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.it"
path_to_original_segment_translations = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.en"
src_lang = "it"
tgt_lang = "en"
path_to_st_model_ckpt = "/kaggle/working/models/joint-s2t-multilingual/checkpoint17.pt"

# Kiểm tra checkpoint tồn tại
if not os.path.exists(path_to_st_model_ckpt):
    print(f"❌ Checkpoint not found: {path_to_st_model_ckpt}")
    exit(1)

working_dir = os.path.dirname(path_to_custom_segmentation_yaml)
segmentation_name = os.path.basename(path_to_custom_segmentation_yaml).replace(".yaml", "")
split_name = os.path.basename(path_to_original_segmentation_yaml).replace(".yaml", "")
st_model_dirname = os.path.dirname(path_to_st_model_ckpt)
st_model_basename = os.path.basename(st_model_dirname)
FAIRSEQ_ROOT = "/kaggle/working/models/fairseq_root"

cmd = [
    "fairseq-generate", working_dir,
    "--task", "speech_text_joint_to_text",
    "--user-dir", f"{FAIRSEQ_ROOT}/examples/speech_text_joint_to_text",
    "--load-speech-only",
    "--gen-subset", segmentation_name,
    "--path", path_to_st_model_ckpt,
    "--max-source-positions", "3700000",
    "--config-yaml", os.path.join(st_model_dirname, "config.yaml"),
    "--infer-target-lang", tgt_lang,
    "--max-tokens", "3700000",
    "--beam", "5"
]

# Thiết lập PYTHONPATH để fairseq có thể import user-dir
env = os.environ.copy()
env["PYTHONPATH"] = FAIRSEQ_ROOT

# Chạy và ghi log
out_path = os.path.join(working_dir, "translations.txt")
print("Running fairseq-generate...")
print("Command:", " ".join(cmd))
with open(out_path, "w") as fout:
    subprocess.run(cmd, stdout=fout, stderr=subprocess.STDOUT, env=env)

print(f"✓ Done. Output written to: {out_path}")
""")
!/kaggle/working/bin/micromamba run -p /kaggle/working/env python run_generate_translation.py


In [ ]:
%cd /kaggle/working/
with open("run_format_generation_output.py", "w") as f:
    f.write("""
import os
import sys
import subprocess

def run_format_generation_output():
    sys.path.append("/kaggle/working/models/fairseq_root")

    # Biến đầu vào
    path_to_wavs = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/wav"
    path_to_custom_segmentation_yaml = "/kaggle/working/results/custom_segments.yaml"
    path_to_original_segmentation_yaml = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.yaml"
    path_to_original_segment_transcriptions = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.it"
    path_to_original_segment_translations = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.en"
    src_lang = "it"
    tgt_lang = "en"
    path_to_st_model_ckpt = "/kaggle/working/models/joint-s2t-multilingual/checkpoint17.pt"


    working_dir = os.path.dirname(path_to_custom_segmentation_yaml)
    segmentation_name = os.path.basename(path_to_custom_segmentation_yaml).replace(".yaml", "")
    split_name = os.path.basename(path_to_original_segmentation_yaml).replace(".yaml", "")
    st_model_dirname = os.path.dirname(path_to_st_model_ckpt)
    st_model_basename = os.path.basename(st_model_dirname)
    FAIRSEQ_ROOT = "/kaggle/working/models/fairseq_root"
    SHAS_ROOT = "/kaggle/working/SHAS"

    # Gọi lệnh format_generation_output.py bằng subprocess
    format_script = os.path.join(SHAS_ROOT, "src/eval_scripts/format_generation_output.py")
    output_path = os.path.join(working_dir, "translations.txt")

    if not os.path.exists(output_path):
        print(f"❌ translations.txt chưa tồn tại tại {output_path}")
        sys.exit(1)

    print(f"✅ Running format_generation_output.py...")
    result = subprocess.run(["python", format_script, "-p", output_path], check=True)
    print("✅ Done.")

if __name__ == "__main__":
    run_format_generation_output()
""")
!/kaggle/working/bin/micromamba run -p /kaggle/working/env python /kaggle/working/run_format_generation_output.py

In [ ]:
%cd /kaggle/working/
with open("run_original_segmentation_to_xml.py", "w") as f:
    f.write("""
    # /kaggle/working/run_eval.py

import os
import sys
import subprocess

def run_original_segmentation_to_xml():
    sys.path.append("/kaggle/working/models/fairseq_root")

    # Biến đầu vào
    path_to_wavs = "//kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/wav"
    path_to_custom_segmentation_yaml = "//kaggle/working/results/custom_segments.yaml"
    path_to_original_segmentation_yaml = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.yaml"
    path_to_original_segment_transcriptions = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.it"
    path_to_original_segment_translations = "/kaggle/input/shas-input/MTEDX_ROOT/MTEDX_ROOT/it-en/data/test/txt/test.en"
    src_lang = "it"
    tgt_lang = "en"
    path_to_st_model_ckpt = "/kaggle/working/models/joint-s2t-multilingual/checkpoint17.pt"

    working_dir = os.path.dirname(path_to_custom_segmentation_yaml)
    segmentation_name = os.path.basename(path_to_custom_segmentation_yaml).replace(".yaml", "")
    split_name = os.path.basename(path_to_original_segmentation_yaml).replace(".yaml", "")
    st_model_dirname = os.path.dirname(path_to_st_model_ckpt)
    st_model_basename = os.path.basename(st_model_dirname)
    FAIRSEQ_ROOT = "/kaggle/working/models/fairseq_root"
    SHAS_ROOT = "/kaggle/working/SHAS"

    # Thiết lập PYTHONPATH
    env = os.environ.copy()
    env["PYTHONPATH"] = FAIRSEQ_ROOT

    # Gọi script eval
    script_path = os.path.join(SHAS_ROOT, "src", "eval_scripts", "original_segmentation_to_xml.py")
    command = [
        "python", script_path,
        "-y", path_to_original_segmentation_yaml,
        "-s", path_to_original_segment_transcriptions,
        "-t", path_to_original_segment_translations,
        "-o", working_dir
    ]

    print("📥 Running original_segmentation_to_xml...")
    subprocess.run(command, env=env, check=True)
    print("✅ Done!")

if __name__ == "__main__":
    run_original_segmentation_to_xml()
    """)
!/kaggle/working/bin/micromamba run -p /kaggle/working/env python /kaggle/working/run_original_segmentation_to_xml.py